In [4]:
import torch
from torch import nn
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l


In [5]:
#多输入通道
def corr2d_multi_in(X,K):
    res = 0
    for i in range(0,X.shape[0]):
        res += d2l.corr2d(X[i,:,:],K[i,:,:])
    return res

In [7]:
#test
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
        [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [8]:
#多输出通道
def corr2d_multi_in_out(X,K):
    #K是stack
    return torch.stack([corr2d_multi_in(X,k) for k in K])

In [9]:
#test
#构造一个三通道的卷积核，用stack存储，每个卷积核通道都是一个三维数组
K = torch.stack([K, K + 1, K + 2])
K.shape

torch.Size([3, 2, 2, 2])

In [10]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [11]:
#1*1 卷积层
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X) # 全连接层的矩阵乘法
    return Y.view(c_o, h, w)

In [12]:
#test
#做1*1卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
(Y1 - Y2).norm().item() < 1e-6

True